In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Loop through data files
for i in range(2, 71):
    # File name based on file numbering
    if i <= 9:
        filename = "data-0" + str(i)
    elif i != 29 and i != 51:
        filename = "data-" + str(i)
    else:
        continue

    # Read data and split columns
    data = pd.read_fwf(filename)
    data[['Date', 'Time', 'Category', 'Target']] = data['Date\tTime\tCategory\tTarget'].str.split('\t', expand=True)
    data.drop(columns=['Date\tTime\tCategory\tTarget'], inplace=True)

    # Define functions for time and date extraction
    def split_time(time_str):
        hours, minutes, seconds = time_str.split(':')
        return int(hours), int(minutes), int(seconds)

    def split_date(date_str):
        date, month, year = date_str.split('-')
        return int(date), int(month), int(year)

    # Process time and date columns
    data[['Hours', 'Minutes', 'Seconds']] = data['Time'].apply(lambda x: pd.Series(split_time(x)))
    data.drop(['Time'], axis=1, inplace=True)
    data[['date', 'Month', 'year']] = data['Date'].apply(lambda x: pd.Series(split_date(x)))
    data.drop(['Date', 'year', 'Month'], axis=1, inplace=True)

    # Create "time" feature and drop unnecessary columns
    data['time'] = (data['Hours'] * 60) + data['Minutes']
    data.drop(['Hours', 'Minutes', 'Seconds', 'date'], axis=1, inplace=True)

    # Feature selection and target assignment
    data = data.dropna()
    X_new = data.iloc[:, :-1].values  # Extracting features
    y_new = data.iloc[:, -1].values  # Extracting target
    y_new = y_new.reshape(-1, 1)  # Reshaping target for scaling

    # Scale features and target
    scaler = MinMaxScaler()
    X_new_scaled = scaler.fit_transform(X_new)
    y_new_scaled = scaler.fit_transform(y_new)

    # Reshape for LSTM input format
    n_features = X_new.shape[1]
    n_obs = X_new.shape[0]
    X_new_reshaped = X_new_scaled.reshape((n_obs, 1, n_features))

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(1, n_features)))  # Adjust units and layers as needed
    model.add(LSTM(units=50))
    model.add(Dense(units=1))

    # Compile and train the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_new_reshaped, y_new_scaled, epochs=10, batch_size=32)

    # Make predictions and inverse transform
    predictions = model.predict(X_new_reshaped)
    predictions = scaler.inverse_transform(predictions)

    # Plot predictions and target values (uncomment if needed)
    plt.scatter(range(len(predictions)), predictions, label="Predictions")
    plt.scatter(range(len(predictions)), y_new, label="Actual Values")
    plt.legend()
    plt.xlabel("Data Points")
    plt.ylabel("Sugar Value")
    plt.show()

print("Finished processing all data files")


AttributeError: Exception encountered when calling layer "lstm_10" (type LSTM).

module 'tensorflow._api.v2.compat.v2.__internal__.function' has no attribute 'defun_with_attributes'

Call arguments received by layer "lstm_10" (type LSTM):
  • inputs=tf.Tensor(shape=(None, 1, 2), dtype=float32)
  • mask=None
  • training=None
  • initial_state=None